# 1. Tri de la base de données

Pré-requis : 
- Avoir téléchargé la base de données sous le nom "archive" et l'avoir placé à la même racine que le repository
- Télécharger les dépendances ci-dessous

Le script suivant permet de réaliser un premier tri de notre base de données. La database originale contient une grande quantité d'images vides, ou bien seulement avec une petite partie d'image satellite. Il faut également par la suite vérifier que les images restantes sont utiles pour entraîner notre algorithme. En effet, une image qui ne contient aucun corps d'eau, ou aucune étendue inondable, ne sera pas utile dans la phase d'apprentissage.

In [ ]:
#### BIBLIOTHEQUES NECESSAIRES ####

# Pour installer la librairie scikit-image, décommenter la ligne suivante :
# pip install scikit-image

import os
import numpy as np
from skimage import io, color

In [ ]:
#### FONCTION DE FILTRAGE ####

# Cette fonction permet de filtrer les images de la base de données en fonction de leur contenu. Le seuil (threshold) est fixé à 0.75 par défaut, mais peut être modifié si besoin.

def is_relevant_image(image_path, threshold=0.75):
    """
        This script checks that the number of pixels containing a grey value (0<x<1) is higher than the threshold. If this is not the case, the image
        is considered to contain too few useful data, and is not saved in the final database.

        Returns a mask
    """
    # Lecture de l'image
    img = io.imread(image_path)
    
    # On transforme en grayscale si l'image est en couleur
    if len(img.shape) == 3:
        img = color.rgb2gray(img)

    # Creation d'un masque pour les pixels qui ne sont ni noirs ni blancs (valeurs de pixels entre 0 et 1)
    gray_pixel_mask = (img > 0) & (img < 1)

    # Calcul du nombre total de pixels
    total_pixels = gray_pixel_mask.size
    
    # Calcul du nombre de pixels gris
    gray_pixel_count = np.sum(gray_pixel_mask)
    
    # Calcul de la proportion de pixels gris
    proportion_gray = gray_pixel_count / total_pixels
    
    # On vérifie si la proportion de pixels gris est supérieure au seuil
    return proportion_gray >= threshold

def filter_images(input_folder, output_folder):
    # On crée les dossiers de sortie s'ils n'existent pas
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(f'{output_folder}/vh', exist_ok=True)
    os.makedirs(f'{output_folder}/vv', exist_ok=True)
    os.makedirs(f'{output_folder}/water_body_label', exist_ok=True)
    os.makedirs(f'{output_folder}/flood_label', exist_ok=True)

    # On parcourt les images de la base de données
    for filename in os.listdir(f'{input_folder}/vh'):   # On parcourt les images du dossier vh pour regarder si l'image est pertinente
        file_path_vh = os.path.join(f'{input_folder}/vh', f'{filename[:-7]}_vh.png')    # On utilise les f-strings de python pour modifier les noms de fichier dans la db
        file_path_vv = os.path.join(f'{input_folder}/vv', f'{filename[:-7]}_vv.png')
        file_path_water = os.path.join(f'{input_folder}/water_body_label', f'{filename[:-7]}.png')
        file_path_flood = os.path.join(f'{input_folder}/flood_label', f'{filename[:-7]}.png')
        
        if os.path.isfile(file_path_vh):
            # On vérifie si l'image est pertinente
            if is_relevant_image(file_path_vh):
                # On sauvegarde les images pertinentes
                img_vh = io.imread(file_path_vh)
                io.imsave(os.path.join(f'{output_folder}/vh', f'{filename[:-7]}_vh.png'), img_vh)
                #print(f'Saved: {filename}')

                img_vv = io.imread(file_path_vv)
                io.imsave(os.path.join(f'{output_folder}/vv', f'{filename[:-7]}_vv.png'), img_vv)

                img_water = io.imread(file_path_water)
                io.imsave(os.path.join(f'{output_folder}/water_body_label', f'{filename[:-7]}.png'), img_water)

                img_flood = io.imread(file_path_flood)
                io.imsave(os.path.join(f'{output_folder}/flood_label', f'{filename[:-7]}.png'), img_flood)

            else:
                continue
                #print(f'Rejected: {filename}')

## 2. On choisis ensuite les dossiers d'images à traiter

In [ ]:
#### CHOIX DES DOSSIERS D'IMAGES ####

# Pour charger 3 dossiers d'images :
#folders = ['bangladesh_20170314t115609','bangladesh_20170606t115613','bangladesh_20170712t115615']

#Pour charger 1 seul dossier d'images :
folders = ['bangladesh_20170314t115609']

## 3. Filtrage : création d'un dossier contenant les images sélectionnées respectant le critère de "grey_scale"

Une fois l'ensemble du code exécuté, un dossier "Final_Database" sera créé contenant 3 sous dossiers : vv, vh et water_body_label

Ces dossiers seront utiles lors de l'apprentissage

In [ ]:
#### FILTRAGE DES IMAGES ####

# !! ADAPTER LES CHEMINS SELON BESOINS !!

for folder in folders:
    input_folder = f'../archive/train/train/{folder}/tiles'
    output_folder = f'../Final_database'
    filter_images(input_folder, output_folder)

/tmp/ipykernel_6109/2054490488.py:57: UserWarning: ../Final_database/water_body_label/bangladesh_20170314t115609_x-8_y-8.png is a low contrast image
  io.imsave(os.path.join(f'{output_folder}/water_body_label', f'{filename[:-7]}.png'), img_water)
/tmp/ipykernel_6109/2054490488.py:60: UserWarning: ../Final_database/flood_label/bangladesh_20170314t115609_x-8_y-8.png is a low contrast image
  io.imsave(os.path.join(f'{output_folder}/flood_label', f'{filename[:-7]}.png'), img_flood)
/tmp/ipykernel_6109/2054490488.py:57: UserWarning: ../Final_database/water_body_label/bangladesh_20170314t115609_x-37_y-27.png is a low contrast image
  io.imsave(os.path.join(f'{output_folder}/water_body_label', f'{filename[:-7]}.png'), img_water)
/tmp/ipykernel_6109/2054490488.py:60: UserWarning: ../Final_database/flood_label/bangladesh_20170314t115609_x-37_y-27.png is a low contrast image
  io.imsave(os.path.join(f'{output_folder}/flood_label', f'{filename[:-7]}.png'), img_flood)
/tmp/ipykernel_6109/20544904